## 밀집 표현 (Dense Representation)
* 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞춤
* 이 과정에서 0과 1만 가진 값이 아니라 실수값을 가지게 됨
* 예
    * 강아지 = [0 0 0 1 .... 0] # 10,000차원
    * 밀집 표현으로 변경
    * 강아지 = [0.2 1.8 1.1 ...] # 원하는 128차원으로 변경 가능
    * 벡터의 차원이 조밀해졌다고 해서 밀집 벡터 (Dense vector)라고 함

## 단어 임베딩 
* 공간적 낭비를 해결하기 위한 방법으로 
* 단어의 의미를 고려하고 밀집 벡터의 형태로 표현 
* 의미가 비슷한 단어는 비슷한 방향에 위치
* 단어의 의미를 효과적으로 표현하기 때문에 원-핫 인코딩보다 학습 성능을 높일 수 있음
* 주어진 배열을 정해진 길이로 압축
* 예
    * 단어 happy는 bad보다 good에 더 가깝고
    * Cat은 Good보다는 Dog에 가깝다는 것을 고려하여 배열을 새로운 수치로 변경

## 케라스 임베딩 구현
* 임베딩 구현
* model = Sequential()
* model.add(Embedding(16,4, input_length=2))
* 단어 -> 정수값 변환 -> 임베딩 층 통과 -> 밀집 벡터

* 임베딩 층
    * 입력 정수에 대해 밀집 벡터로 맵핑하고
    * 밀집 벡터는 인공 신경망의 학습 과정에서 가중치가 학습되는 것과 같은 방식으로 훈련
    * 훈련 과정에서 단어는 모델이 해결하고자 하는 작업에 맞는 값으로 업데이트 됨
    * 이 밀집 벡터를 임베딩 벡터라고 함
* 임베딩 층의 출력 크기
* model.add(Embedding(16,4, input_length=2)) : 3D 실수형 텐서를 반환

* Flatten층 추가 : 3D 실수형 텐서를 2D로 변환
    * 다음의 Dense()에서 처리할 수 있는 입력으로 변환
    * model.add(Flatten())

## 패딩 (Padding)
* 문장의 길이를 나타내는 배열의 크기를 동일하게 맞추는 작업
* 길이가 전부 동일한 문서들에 대하여 하나의 행렬로 인식하고
* 한꺼번에 묶어서 처리할 수 있도록 하기 위함
* 각 문장 (문서)의 길이가 서로 다른 경우
* 동일한 길이로 맞추고 빈 부분은 0으로 채운다 

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ['지금은 딥러닝을 공부하고 있어요','어렵지 않고 쉬워요','참 재미있어요']

token = Tokenizer()
token.fit_on_texts(sentence)
print(token.word_index)

{'지금은': 1, '딥러닝을': 2, '공부하고': 3, '있어요': 4, '어렵지': 5, '않고': 6, '쉬워요': 7, '참': 8, '재미있어요': 9}


In [4]:
# 각 문장에 해당되는 인덱스를 배열로 생성
result = token.texts_to_sequences(sentence)
result

[[1, 2, 3, 4], [5, 6, 7], [8, 9]]

In [9]:
from keras.preprocessing.sequence import pad_sequences

In [10]:
# 패딩 : 각 배열의 길이를 동일하게 맞춤
# 길이를 지정하지 않으면 제일 긴 배열의 크기에 맞춰준다
padded = pad_sequences(result)
padded

array([[1, 2, 3, 4],
       [0, 5, 6, 7],
       [0, 0, 8, 9]], dtype=int32)

In [11]:
padded = pad_sequences(result, padding='post',maxlen=6) # 길이를 6으로 지정 (Maxlen=6)
padded

array([[1, 2, 3, 4, 0, 0],
       [5, 6, 7, 0, 0, 0],
       [8, 9, 0, 0, 0, 0]], dtype=int32)

# 자연어 처리 예제

* 텍스트를 읽고 긍정, 부정 예측하기
* 영화 리뷰를 읽고 딥러닝 모델로 학습해서 
* 각 리뷰가 긍정적인지 부정적인지 예측
* 예측 과정
1. 짧은 리뷰 10개를 불러와서 긍정적이면 1, 부정적이면 0 클래스로 지정
2. 토큰화
3. 패딩
4. 임베딩 및 딥러닝 처리

In [1]:
import numpy as np
import tensorflow as tf
from numpy import array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [2]:
# 1. 짧은 리뷰 10개를 불러와서 긍정적이면 1, 부정적이면 0 클래스로 지정
docs = ["너무 재밌네요","최고예요","참 잘 만든 영화예요", \
        "추천하고 싶은 영화입니다","한번 더 보고싶네요","글쎄요", \
        "별로예요","생각보다 지루하네요","연기가 어색해요","재미없어요"]

classes = array([1,1,1,1,1,0,0,0,0,0])

In [3]:
# 2. 토큰화
token = Tokenizer()
token.fit_on_texts(docs)
print(token.word_index)

{'너무': 1, '재밌네요': 2, '최고예요': 3, '참': 4, '잘': 5, '만든': 6, '영화예요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로예요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}


In [4]:
# 각 문장에 해당되는 인덱스를 배열로 생성 
x = token.texts_to_sequences(docs)
print(x)

[[1, 2], [3], [4, 5, 6, 7], [8, 9, 10], [11, 12, 13], [14], [15], [16, 17], [18, 19], [20]]


In [5]:
# 3. 패딩 : 길이를 4로 맞춤
padded_x = pad_sequences(x,4)
print('패딩 결과 : \n', padded_x)

패딩 결과 : 
 [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


### 딥러닝 모델 적용
1. 모델 설정
2. 모델 컴파일
3. 모델 훈련 (학습)
4. 모델 평가

In [6]:
# 임베잉에 입력할 단어 수 지정
# 맨 앞에 0이 나오도록 len() + 1
word_size = len(token.word_index)+1

In [38]:
# 1. 모델 설정
# 임베딩층 구현
model = Sequential()
model.add(Embedding(word_size,8,input_length=4)) # 3d 텐서
model.add(Flatten())# 2d로 변환
model.add(Dense(1,activation='sigmoid'))

# 2. 모델 컴파일
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])


# 3. 모델 훈련 (학습)
model.fit(padded_x, classes, epochs=20)

# 4. 모델 평가 
print('\n Accuracy : %.4f' %(model.evaluate(padded_x,classes)[1]))

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6890 - accuracy: 0.6000
Epoch 2/20
1/1 [==============================] - 0s 26ms/step - loss: 0.6869 - accuracy: 0.6000
Epoch 3/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6848 - accuracy: 0.6000
Epoch 4/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6826 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6805 - accuracy: 0.7000
Epoch 6/20
1/1 [==============================] - 0s 7ms/step - loss: 0.6784 - accuracy: 0.8000
Epoch 7/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6763 - accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 0s 9ms/step - loss: 0.6742 - accuracy: 0.8000
Epoch 9/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6721 - accuracy: 0.8000
Epoch 10/20
1/1 [==============================] - 0s 17ms/step - loss: 0.6700 - accuracy: 0.8000
Epoch 11/20
1/1 [===================

In [7]:
# 1. 모델 설정
# 임베딩층 구현
model = Sequential()
model.add(Embedding(word_size,8,input_length=4)) # 3d 텐서
model.add(Flatten())# 2d로 변환
model.add(Dense(1,activation='sigmoid'))

# 2. 모델 컴파일
model.compile(optimizer='rmsprop', #rmsprop #Adam
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])


# 3. 모델 훈련 (학습)
model.fit(padded_x, classes, epochs=20)

# 4. 모델 평가 
print('\n Accuracy : %.4f' %(model.evaluate(padded_x,classes)[1]))

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6963 - accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 0s 4ms/step - loss: 0.6897 - accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 0s 10ms/step - loss: 0.6849 - accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6809 - accuracy: 0.6000
Epoch 5/20
1/1 [==============================] - 0s 11ms/step - loss: 0.6774 - accuracy: 0.8000
Epoch 6/20
1/1 [==============================] - 0s 14ms/step - loss: 0.6741 - accuracy: 0.9000
Epoch 7/20
1/1 [==============================] - 0s 8ms/step - loss: 0.6710 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 19ms/step - loss: 0.6680 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 12ms/step - loss: 0.6652 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 29ms/step - loss: 0.6624 - accuracy: 1.0000
Epoch 11/20
1/1 [=================